# Ask Sage Python API Client - API Documentation

In this notebook, we will go through the AskSage Python API Client documentation and review the available endpoints.

For more information please visit the AskSage Python API Client via the link below:
Python API Client: https://pypi.org/project/asksageclient/

<div class="alert alert-block alert-info">
<b>Tip:</b> Recommend to run the code cells in order to see the output of the endpoints and understand the examples provided. 
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> All credential information and API keys are removed from the code cells and declared as environment variables for security purposes. We recommend to do the same when running the code cells if you are sharing the notebook.
</div>

We will cover all of the following endpoints in this notebook: 

The order of review is based on a logical flow of first understanding what the API has to offer, then interacting with the API to interact with models. 

Also, refer to the Outline of the notebook to navigate to the specific endpoints you are interested in reviewing.

|       Function Name         |                       Description                     |
|:---------------------------:|:-----------------------------------------------------:|
|         `get_models`         | Get the available models from the Ask Sage service    |
|       `add_dataset`         |                   Adds a new dataset                  |
|     `delete_dataset`        |              Deletes a specified dataset              |
|     `assign_dataset`        |                   Assigns a dataset                   |
|     `get_user_logs`         |             Retrieves all logs for user               |
|    `get_user_logins`        | Retrieves login information for a specific user       |
|          `query`            | Interact with the /query endpoint of the Ask Sage API |
|    `query_with_file`        |         Executes a query using a file                 |
|      `query_plugin`         | Executes a query using a specific plugin              |
|    `execute_plugin`         | Executes a plugin with the provided content           |
| `follow_up_questions`       | Interact with the /follow-up-questions endpoint of the Ask Sage API |
|        `tokenizer`          | Interact with the /tokenizer endpoint of the Ask Sage API |
|      `get_personas`         | Get the available personas from the Ask Sage service  |
|      `get_datasets`         | Get the available datasets from the Ask Sage service  |
|       `get_plugins`         | Get the available plugins from the Ask Sage service   |
|  `count_monthly_tokens`     | Get the count of monthly training tokens spent for this user from the Ask Sage service |
|`count_monthly_teach_tokens` | Counts the number of teach tokens used in a month     |
|          `train`            | Train the model based on the provided content         |
|    `train_with_file`        | Train the dataset based on the provided file          |
|          `file`             | Upload a file to the Ask Sage service                 |

## Import Libraries and Set Environment Variables 

Let's start by importing the necessary libraries and setting the environment variables.

> There will be more detail within this notebook than what is relevant to the user, but it is important to understand the different endpoints and what they represent. Also, note that the code is not being written in a production-ready manner, but rather to demonstrate the different endpoints and their outputs.

In [1]:
import json # Import the json module to work with JSON data
import requests # Import the requests library to send HTTP requests
from asksageclient import AskSageClient # Import the AskSageClient class from the asksageclient module
import pandas as pd # Import the pandas library to work with dataframes
import os # Import the os module to interact with the operating system
from PIL import Image # Import the Image class from the PIL module to work with images
from io import BytesIO # Import the BytesIO class from the io module to work with bytes

# Function to load credentials from a JSON file
def load_credentials(filename):
    try:
        with open(filename) as file:
            return json.load(file)
    except FileNotFoundError:
        raise FileNotFoundError("The credentials file was not found.")
    except json.JSONDecodeError:
        raise ValueError("Failed to decode JSON from the credentials file.")

# Load the credentials
credentials = load_credentials('../../credentials.json')

# Extract the API key, and email from the credentials to be used in the API request
api_key = credentials['credentials']['api_key']
email = credentials['credentials']['Ask_sage_user_info']['username']


## Setup Ask Sage Python API Client

The AskSage Python API Client provides a simple way to interact with the AskSage API. The client provides methods for each endpoint, making it easy to use the API.

First order is defining the client and setting the credentials for the client, which are the email and API key.



In [2]:
"""  
class AskSageClient(
    email: email, # The email address of the user
    api_key: api_key, # The API key for the AskSage API, which can be obtained from the AskSage website
    user_base_url: str = 'https://api.asksage.ai/user', # The base URL for the user API
    server_base_url: str = 'https://api.asksage.ai/server' # The base URL for the server API
)
"""

ask_sage_client = AskSageClient(email, api_key) # Create an instance of the AskSageClient class with the email and api_key 

## 1. Get Models
The `get_models` function retrieves all the available models from the Ask Sage services. The models can be called with the `query` function to interact with user queries. However, note that developers need to understand the models and their capabilities as they are different from each other.

In [3]:
models = ask_sage_client.get_models() # Get the list of models available for the user

# Get the list of models available on Ask SAge. 
models = models['response']
print("These are the models available on Ask Sage: " + str(models))


These are the models available on Ask Sage: ['aws-bedrock-titan', 'llma3', 'claude2', 'claude-3-opus', 'claude-3-sonnet', 'claude-35-sonnet', 'cohere', 'mistral-large', 'gpt-gov', 'gpt4-gov', 'gpt', 'gpt4', 'gpt4-32k', 'gpt35-16k', 'gpt4-vision', 'gpt-4o', 'gpt-4o-mini', 'dall-e-2', 'dall-e-3', 'google-bison', 'google-gemini-pro', 'gpt-4o-gov', 'groq-70b', 'gpt-o1', 'gpt-o1-mini', 'xai-grok']


<div class="alert alert-block alert-warning">
<b>Note:</b> Reference the Ask Sage 'Help' section 'Models' Tab to understand the models and their capabilities. Additionally, if the models are CUI compatible.
</div>


## 2. Get User Logs

This endpoint returns the last prompts of the user. By default, it returns the last 5 prompts.

There are no parameters required for this endpoint. 

The output will be the following:
- All the prompts that the user has made.
- completion_tokens: The completion tokens of the prompt.
- date_time: The date and time of the prompt.
- id: The ID of the prompt.
- ip: The IP address of the prompt.
- model: The model used for the prompt.
- prompt: The prompt text.
- prompt_tokens: The prompt tokens.
- response: The response text.
- teach: The teach flag (true or false).
- total_tokens: The total tokens 


<div class="alert alert-block alert-danger">
<b>AskSage - Requested Updated:</b> The endpoint works, but it outputs all everything listed above - a feature request is to be able to limit the number of outputs.
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> Uncomment the `display` call to see the output of the endpoint in the cell below. It was intentionally commented out to avoid showing the output in the notebook. 

Also, be aware that this endpoint outputs all the prompts that the user has made, which may contain sensitive information. Therefore, it is recommended to use this endpoint with caution and avoid displaying the output in a shared notebook.

</div>




In [4]:
user_prompts = ask_sage_client.get_user_logs() # Get the user prompts using the get_user_logs method
# display(user_prompts) # comment to hide the user prompts - this will display a lot of data 

## 3. Get User Logins

This endpoint returns the last logins of the user. By default, it returns the last 5 logins.

The only parameter required for this endpoint is the 'limit' parameter, which specifies the number of logins to return. The default value is 5 and the maximum value is 100.

So in return, one will get the following information: 
- date_time: The date and time of the login.
- id: The ID of the login.
- ip: The IP address of the login.
- status: The status of the login (success or failure).
- type: endpoint type (login).
- status_code: The status code of the login (200 for success, 400 for failure).

<div class="alert alert-block alert-warning">
<b>Note:</b> Uncomment the `display` call to see the output of the endpoint in the cell below. It was intentionally commented out to avoid showing sensitive information in the notebook.
</div>


In [5]:
user_logins = ask_sage_client.get_user_logins(limit=5) # Get the user logins using the get_user_logins method
# display(user_logins) 

## 4. Count Monthly Tokens

Get the count of monthly training tokens spent for this user from the Ask Sage service - This endpoint will return the number of tokens used in the current month.

In [6]:
count_monthly_usage = ask_sage_client.count_monthly_tokens() # Get the count of monthly usage using the get_count_monthly_usage method
# extract the count of monthly usage from the response
count = count_monthly_usage['response']

print(f"The count of monthly usage is: {count}") # Print the count of monthly usage


The count of monthly usage is: 453597


## 5. Count Monthly Teach Tokens

 Get the count of monthly training tokens spent for this user from the Ask Sage service, meaning the number of tokens used for training the model.


In [7]:
count_monthly_usage = ask_sage_client.count_monthly_teach_tokens() # Get the count of monthly usage using the get_count_monthly_usage method
# extract the count of monthly usage from the response
count = count_monthly_usage['response']

print(f"The count of monthly teach usage is: {count}") # Print the count of monthly usage


The count of monthly teach usage is: 907135


## 6. Get Personas

This endpoint returns the available personas from the Ask Sage service. Users will have access to all the standard personas that are available in the Ask Sage service for all paid users, and any custom personas that have been created in the platform. 

<div class="alert alert-block alert-danger">
<b>AskSage - Requested Updated:</b> Being able to create custom personas via the API would be a great feature to have. Although, the `query` endpoint does have a parameter for `system_prompt` which can be used to define a persona.
</div>

In [8]:
get_personas = ask_sage_client.get_personas() # Get the personas using the get_personas endpoint
# output of get_personas is as follows for each persona
""" out put is as follows

{'response': [{'datasets': '',
   'date_creation': 'Tue, 16 Jan 2024 18:35:32 GMT',
   'date_modification': 'Tue, 16 Jan 2024 18:35:32 GMT',
   'description': 'Use this persona when you need a general-purpose AI that can handle a wide range of tasks, from translating languages to writing essays and code.',
   'id': 1,
   'image': None,
   'label': 1,
   'name': 'Ask Sage',
   'prompt': 'Your purpose is help organizations drive outcomes by ingesting knowledge and data, providing analysis and insights with factual answers. \nYou are able to translate languages, write essays, articles, bids, code and more.',
   'public': True,
   'user_id': -1}
"""

# extract relevant information from the response 'id', 'name' and 'description'
def extract_personas(response):
    return [{'id': persona['id'], 'name': persona['name'], 'description': persona['description']} for persona in response['response']]

personas = extract_personas(get_personas) # Extract the personas using the extract_personas function

# Putting all information into a dataframe
personas_df = pd.DataFrame(personas)

# set column width to display full content
pd.set_option('display.max_colwidth', None)

# display the dataframe
display(personas_df.head(15)) # modify .head() to display all or a specific number of rows


,id,name,description
0,1,Ask Sage,"Use this persona when you need a general-purpose AI that can handle a wide range of tasks, from translating languages to writing essays and code."
1,2,Legal Assistant,Use this persona when you need legal advice or information. This persona can provide accurate and helpful advice on a wide range of legal topics.
2,3,Contracting Officer,"Use this persona when you have questions or need advice about government contracts, Federal Acquisition Regulation (FAR) regulations, the Defense Federal Acquisition Regulation Supplement (DFARS) regulations and acquisition related questions."
3,4,Software Developer,"Use this persona when you need to write, review, or debug code. This persona can also provide advice on software development best practices and security standards."
4,5,ISSO (Cyber),Use this persona when you need advice or information about cybersecurity requirements and issues. This persona can provide accurate and helpful advice on a wide range of cybersecurity topics.
5,6,System Administrator,"Use this persona when you need advice or information about system designs, security, and issues. This persona can provide accurate and helpful advice on a wide range of system administration topics."
6,7,Structural Engineer,Use this persona when you need advice or information about structural engineering. This persona can provide accurate and helpful advice on a wide range of structural engineering topics.
7,8,Electrical Engineer,Use this persona when you need advice or information about electrical engineering. This persona can provide accurate and helpful advice on a wide range of electrical engineering topics.
8,9,Accountant,Use this persona when you need advice or information about financial matters. This persona can provide accurate and helpful advice on a wide range of accounting and financial topics.
9,10,DevSecOps Engineer,Use this persona when you need advice or information about DevSecOps and software operations. This persona can provide accurate and helpful advice on a wide range of DevSecOps topics.


## 7. Tokenizer

This endpoint is used to interact with the /tokenizer endpoint of the Ask Sage API. It is used to tokenize the text provided to the model. The tokenizer is used to split the text into tokens that the model can understand and process. The tokenizer is used to preprocess the text before it is passed to the model for processing. Thus in return the output will be the number of tokens from the text provided.

<div class="alert alert-block alert-warning">
<b>Note:</b> The `tokenizer` endpoint is useful in estimating the number of tokens that will be used for a given text. This can be helpful in managing the number of tokens used for training and querying the model.
</div>


In [9]:
text = "This endpoint is used to interact with the /tokenizer endpoint of the Ask Sage API. It is used to tokenize the text provided to the model. The tokenizer is used to split the text into tokens that the model can understand and process. The tokenizer is used to preprocess the text before it is passed to the model for processing. Thus in return the output will be the tokens of the text provided." 

tokenize_data = ask_sage_client.tokenizer(text) # Tokenize the text using the tokenizer method

display(tokenize_data)

# display the field 'response'
display('The number of tokens in the text is: ' +
        tokenize_data['response'] + ' Tokens') # Display the response from the API, and check if the dataset was added successfully on the AskSage website

{'response': '84', 'status': 200}

'The number of tokens in the text is: 84 Tokens'

## 8. File

This is a function named file in the Ask Sage service. It is used to upload a file to the service. The function takes two parameters: file_path and strategy.

file_path is a string that represents the path to the file you want to upload.

strategy is also a string, and it determines the type of parser that will be used. By default, it is set to 'auto'. If you want faster parsing but less accuracy, you can set it to 'fast'. If you need OCR recognition, you can set it to 'hi_res', but keep in mind that this will be slower.

The function opens the file in binary mode and prepares it for upload. It then makes a POST request to the 'file' endpoint of the service, passing the file and the strategy as data.

The function returns a dictionary which is the response from the service, containing the text/plain of the uploaded file.

<div class="alert alert-block alert-warning">
<b>Note:</b> The `file` endpoint is useful in being able to parse a file and then use `Train` endpoint to pass through the content and assign it to a dataset within Ask Sage. 

Also, understanding how this endpoint works is important in understanding how data is ingested into the Ask Sage service.
</div>



In [10]:
# data path
path = 'data/query_with_file/'

# get files in the data path
files = os.listdir(path)
display(files)
file_endpoint = ask_sage_client.file(file_path = path + files[1], strategy='auto') # change the index between 0 and 1 to test the other file and see how it parses the data into chunks for large files. 
# display the field 'response'
display(file_endpoint)

# extract the 'ret' field from the response
ret = file_endpoint['ret']
# count # of chunks by the number of 'asksage_metadata' appears in ret
count = ret.count('asksage_metadata')
print(f'The number of chunks is: {count}')

['LORA- LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS.pdf',
 'In the small village of Willowbrook.pdf']

{'response': 'OK',
 'ret': '\n{"asksage_metadata": {"filename": "In the small village of Willowbrook.pdf", "page_number": 1}}\nIn the small village of Willowbrook, there lived a scruffy little dog named Toby who had an unusual dislike for cats. Toby, with his ruffled fur and eager eyes, was known around the village for his playful spirit, but he would always steer clear of any feline. One day, a new cat moved into the neighborhood. The cat, sleek and confident, quickly noticed Toby\'s aversion. Curious and a bit mischievous, the cat decided to find out why Toby was so hesitant around its kind. This set the stage for a series of amusing encounters, as the cat tried to win over the reluctant Toby, leading to unexpected friendship and adventures that would change Toby\'s mind about cats forever.\n',
 'sent_filename': 'InthesmallvillageofWillowbrook.pdf',
 'status': 200}

The number of chunks is: 1


## 9. Get Datasets

Get the available created datasets datasets from the Ask Sage service - These datasets are used to interact with the LLMs models. More information will be provided on how to interact with the datasets in the following endpoints & examples in the repository.

Note: The following datasets are available by default to all users:


<center>

| Index | Dataset Name              |
|-------|---------------------------|
| 0     | Acquisition.gov           |
| 1     | Air Force                 |
| 2     | DoD                       |
| 3     | Department of Defense     |
| 4     | Learn with Nic            |
| 5     | In the Nic of Time        |
| 6     | Platform One              |
| 7     | Nic Chaillan's Website    |
| 8     | Cloud One                 |
| 9     | NIST_NVD_CVE              |
| 10    | Sage                      |
| 11    | user_custom_2780_content  |
</center>

<div class="alert alert-block alert-warning">
<b>Note:</b> The `user_custom_2780_content` dataset is a unique dataset to each user - based on the user ID. Not typically used and can not be deleted. Nor can the other default datasets be deleted.
</div>


In [11]:
# extract relevant information from the response
def extract_datasets(response):
    return response['response']

get_datasets = ask_sage_client.get_datasets() # Get the datasets using the get_datasets endpoint


def display_datasets(ask_sage_client):
    """   
    Function to display the datasets in a dataframe

    Parameters:
    ask_sage_client: AskSageClient - The AskSageClient instance

    Returns:
    None - Displays the datasets in a dataframe
    
    """
    get_datasets = ask_sage_client.get_datasets() # Get the datasets using the get_datasets endpoint
    datasets = extract_datasets(get_datasets) # Extract the datasets using the extract_datasets function
    datasets_df = pd.DataFrame(datasets)
    display(datasets_df) # Display the updated datasets dataframe

# call the function to display the datasets
display_datasets(ask_sage_client)


,0
0,Acquisition.gov
1,Air Force
2,DoD
3,Department of Defense
4,Learn with Nic
5,In the Nic of Time
6,Platform One
7,Nic Chaillan's Website
8,Cloud One
9,NIST_NVD_CVE


## 10. Add Dataset

This endpoint is used to create a new dataset that will be available for the user to use in the prompt generation. Additionally, once the dataset is created it will also be available on the AskSage platform for the user to use. 

Notice how the dataset table now has 12 datasets, with the newly created dataset at index 12.

In [12]:
# Defining a dataset to be added to the user's datasets
add_dataset_data = ask_sage_client.add_dataset('test-test-test-01') # Replace 'test-test-test' with the name of the dataset you want to add
print(add_dataset_data) # Display the response from the API, and check if the dataset was added successfully 
display_datasets(ask_sage_client) # Display the datasets after adding a new dataset


{'response': 'OK', 'status': 200}


,0
0,Acquisition.gov
1,Air Force
2,DoD
3,Department of Defense
4,Learn with Nic
5,In the Nic of Time
6,Platform One
7,Nic Chaillan's Website
8,Cloud One
9,NIST_NVD_CVE


## 11. Train

The train endpoint is used to train the model based on the provided content. The content is the message to be processed by the service. Ensure it is under 500 tokens. The force_dataset is the dataset to be used. Enter your custom dataset, must follow the following format: user_content_USERID_DATASET-NAME_content. Replace USERID by user ID and DATASET-NAME by the name of your dataset. The context is a short context about the content (metadata). Under 20 tokens. The skip_vectordb is whether to skip the VectorDB training. Default is False.

In [13]:
content = 'Arduino is an open-source electronics platform based on easy-to-use hardware and software. It consists of a physical programmable circuit board (often referred to as a microcontroller) and a piece of software, or IDE (Integrated Development Environment), that runs on your computer. You use the IDE to write and upload computer code to the physical board. The platform is designed to enable users of all ages to create interactive electronic objects and projects that can sense and control physical devices. Arduino boards can read inputs - light on a sensor, a finger on a button, or a Twitter message - and turn it into an output - activating a motor, turning on an LED, publishing something online. You can tell your board what to do by sending a set of instructions to the microcontroller on the board. Arduino is widely used in robotics, home automation, scientific experimentation, and artistic projects.'

tokenize_data = ask_sage_client.tokenizer(content) # Tokenize the text using the tokenizer method

# display the field 'response'
display('The number of tokens in the text is: ' +
        tokenize_data['response'] + ' Tokens') # Display the response from the API, and check if the dataset was added successfully on the AskSage website


'The number of tokens in the text is: 179 Tokens'

In [14]:
# train the content into the database
train_data = ask_sage_client.train(content, force_dataset='test-test-test', context='testing') # Replace 'test-test-test' with the name of the dataset you want to add
# display the field 'response'
display(train_data['response']) # Display the response from the API, and check if the dataset was added successfully on the AskSage website

'Sorry, this dataset (test-test-test) does not exist, please create it with /add-dataset first.'

<div class="alert alert-block alert-danger">
<b>AskSage - Requested Updated:</b> When selecting a dataset, the instruction mention to use the following format: user_content_USERID_DATASET-NAME_content. However, the dataset is not being recognized when using the format provided. Just used the dataset name and it worked. Also, successful uploading the data but it's not appearing on the webiste. 
</div>

## 12. Train With File

This endpoint is used to train the dataset based on the provided file. The files that can be loaded are listed below: 

- Format supported: zip, pdf, xlsx, pptx, docx, ppt, csv, cc, sql, cs, hh, c, php, js, py, html, xml, msg, odt, epub, eml, rtf, txt, doc, json, md, jpeg, jpg, png, tsv (50MB)

- Audio Format supported: mp3, mp4, mpeg, mpga, m4a, wav, webm (500MB max)

The files uploaded will be vectorized and only the embeddings will be stored. The files will not be stored on the server.

In [15]:
'''  
How the add_dataset method works:

 def train_with_file(self, file_path, dataset):
        """
    Train the dataset based on the provided file.

    Parameters:
    file_path (str): The file to upload to the service.
    dataset (str): The dataset to be used. Enter your custom dataset, must follow the following format: user_content_USERID_DATASET-NAME_content. Replace USERID by user ID and DATASET-NAME by the name of your dataset.
    
    Returns:
    dict: The response from the service.
        """
        with open(file_path, 'rb') as f:
            files = {'file': f}
            return self._request('POST', 'train-with-file', files=files, data={'dataset': dataset})
'''

# data path
file_path = 'data/' 

# get files in the data path
files = os.listdir(file_path)

# ignore 'data/query_with_file' and hidden files
files = [file for file in files if not file.startswith('.') and file != 'query_with_file']


# train the dataset with the files in the data path                           

for file in files:
    train_with_file_data = ask_sage_client.train_with_file(file_path + file, 'user_custom_2780_test-test-test-01_content') # Replace the dataset with the name of the dataset you are adding files to
    # display the field 'response'
    display(train_with_file_data['response'] + ' for file ' + file) # Display the response from the API, and check if the dataset was added successfully on the AskSage website


'Successfully imported for file training_image_example.jpg'

'Successfully imported for file random_story_genAI_pdf.pdf'

'Successfully imported for file random_story_genAI_word_doc.docx'

'Successfully imported for file Ask_Sage_Intro.mp3'

Data upload successfully, will be stored however there is a lag on availability, which varies based pom file size.

<div class="alert alert-block alert-danger">
<b>AskSage - Requested Updated:</b> Uploaded data however noticed that only word documents and pdfs are actually being uploaded and seen on the platform following the upload.
</div>

## 13. Assign Dataset

This endpoint is used to assign a dataset to a specific user - This will allow another user to use the dataset but only sharing between users is permitted if they are from the same organization.

<div class="alert alert-block alert-warning">
<b>Note:</b> The `assign_dataset` endpoint is useful in being able to share datasets between users within the same organization. 
</div>


In [16]:
''' 
How the assign_dataset method works:

def assign_dataset(self, dataset, email):
    """
    Assign a dataset

    Parameters:
    dataset (str): The dataset to be used. Must follow the following format: user_content_USERID_DATASET-NAME_content. Replace USERID by user ID and DATASET-NAME by the name of your dataset.
    email (str): Email of the user to assign the dataset to. Must be in the same organization. Reach out to support if need be.

    Returns:
    dict: The response from the service.
    """
    return self._request('POST', 'assign-dataset', json={'dataset': dataset, 'email': email}, base_url=self.user_base_url)
'''

# Assign the dataset to the user
# assign_dataset_data = ask_sage_client.assign_dataset('user_content_2780_test-test-test_content', 'email@email.com') # Replace the dataset and the email with who you want to assign the dataset to 

print('Uncomment the above to test the assign_dataset method') # Uncomment the above line to test the assign_dataset method

Uncomment the above to test the assign_dataset method


## 14. Query

This endpoint is used to interact with the /query endpoint of the Ask Sage API. It is where users can interact with the various models available on the Ask Sage platform.

We will provide examples of how to use these endpoints and various parameters that can be used to interact with the models. Do note that this will only be high-level examples.


```python
'''
How the query method works:

def query(self, message, persona='default', dataset='all', limit_references=None, temperature=0.0, live=0, model='openai_gpt', system_prompt=None, file=None, tools=None, tool_choice=None):
        """
    Interact with the /query endpoint of the Ask Sage API.

    Parameters:
    message (str): The message to be processed by the service. Message can be a single message or an array of messages following this JSON format: [{ user: "me", message: "Who is Nic Chaillan?"}, { user: "gpt", message: "Nic Chaillan is..."}]
    persona (str, optional): The persona to be used. Default is 'default'. Get the list of available personas using get_personas.
    dataset (str, optional): The dataset to be used. Default is 'all'. Other options include 'none' or your custom dataset, must follow the following format: user_content_USERID_DATASET-NAME_content. Replace USERID by user ID and DATASET-NAME by the name of your dataset.
    limit_references (int, optional): The maximum number of references (embeddings) to be used. Default is None, meaning all references will be used. Use 1 to limit to 1 reference or 0 to remove embeddings. You can also set dataset to "none"
    temperature (float, optional): The temperature to be used for the generation. Default is 0.0. Higher values (up to 1.0) make the output more random.
    live (int, optional): Whether to use live mode. Default is 0. Live = 1 will pull 10 results from Bing and 2 will also pull the top 2 web pages summaries using our Web crawler.
    model (str, optional): The model to be used. Default is 'openai_gpt'. Other options include cohere, google-bison, gpt4, gpt4-32k, gpt35-16k, claude2, openai_gpt (gpt3.5), davinci, llma2.
    system_prompt (str, optional): Overrides the system prompt from Ask Sage (only use if you know what you are doing).
    tools and tool_choice (optional): These use OpenAI format for tools.

    Returns:
    dict: The response from the service.
    """
        file_obj = None
        files = None
        if file != None:
            file_obj = open(file, 'rb')
            files = {'file': file_obj}

        if type(message) == list:
            message = json.dumps(message)
        elif type(message) == str:
            message = message
        else:
            message = json.dumps(message)

        if tools != None:
            tools = json.dumps(tools)
        if tool_choice != None:
            tool_choice = json.dumps(tool_choice)

        data = {
            'message': message,
            'persona': persona,
            'dataset': dataset,
            'limit_references': limit_references,
            'temperature': temperature,
            'live': live,
            'model': model,
            'system_prompt': system_prompt,
            'tools': tools,
            'tool_choice': tool_choice
        }

        ret = self._request('POST', 'query', files = files, data=data)        
        if file_obj != None:
            file_obj.close()
        return ret

'''

```


#### 'message'

- 'message': The message is your prompt that you want to generate a response for by the model. In other words, the message/question you want to ask the model. But remember, the model will only generate a response based on the data it has been trained on and given this understanding, it is important to ask questions that are relevant to the data the model has been trained on. 
  
If you are using a off-the-shelf model or Out of the Box model, then the model has been trained on a wide range of data and can generate responses to a wide range of questions. However, do not expect the model to generate responses to questions that are not relevant to the data it has been trained on or be an expert in any or all fields. We will discuss more examples on how to direct the model to generate responses to specific questions later. Last but not least, also be aware that each model has its own limitations and capabilities and designed for specific use cases, thus making some more suitable for certain tasks like code generation, text generation, etc.

In [17]:
def ask_sage_question(message, persona='default', dataset=None, limit_references=None, temperature=0.0, live=0, model='gpt-4o-mini', system_prompt=None):
    """
    Function to query the AskSage API with a question and return the response message using one of the personas available

    Parameters:
    message (str): The question to be queried
    persona (str): The persona to be used. Default is 'default'. Get the list of available personas using get_personas.
    dataset (str): The dataset to be used. Default is 'all'. Other options include 'none' or your custom dataset, must follow the following format: user_content_USERID_DATASET-NAME_content. Replace USERID by user ID and DATASET-NAME by the name of your dataset.
    limit_references (int): The maximum number of references (embeddings) to be used. Default is None, meaning all references will be used. Use 1 to limit to 1 reference or 0 to remove embeddings. You can also set dataset to "none"
    temperature (float): The temperature to be used for the generation. Default is 0.0. Higher values (up to 1.0) make the output more random.
    live (int): Whether to use live mode. Default is 0. Live = 1 will pull 10 results from Bing and 2 will also pull the top 2 web pages summaries using our Web crawler.
    model (str): The model to be used. Default is 'openai_gpt'. Other options include cohere, google-bison, gpt4, gpt4-32k, gpt35-16k, claude2, openai_gpt (gpt3.5), davinci, llma2.
    system_prompt (str): Overrides the system prompt from Ask Sage (only use if you know what you are doing).
    tools and tool_choice (optional): These use OpenAI format for tools.


    Returns:
    str: The response message from the AskSage API
    """
    response = ask_sage_client.query(message, persona, dataset, limit_references, temperature, live, model, system_prompt) # Query the AskSage API with the question
    message = response['message'] # Extract the message from the response
    return message # Return the message


In [18]:
# Let's review all of the bad examples
examples_massage = 'How many planets are in the solar system and can you provide the names of the planets in order from the sun? Also, provide the distance of each planet from the sun in kilometers.'

response = ask_sage_question(message=examples_massage)
print(f"Question: {examples_massage}")
print(f"Response: {response}")

Question: How many planets are in the solar system and can you provide the names of the planets in order from the sun? Also, provide the distance of each planet from the sun in kilometers.
Response: There are eight planets in the solar system. Here are their names in order from the Sun, along with their average distances from the Sun in kilometers:

| Planet       | Average Distance from the Sun (km) |
|--------------|------------------------------------|
| Mercury      | 57,910,000                         |
| Venus        | 108,200,000                        |
| Earth        | 149,600,000                        |
| Mars         | 227,900,000                        |
| Jupiter      | 778,500,000                        |
| Saturn       | 1,429,000,000                      |
| Uranus       | 2,871,000,000                      |
| Neptune      | 4,495,000,000                      |

These distances can vary slightly due to the elliptical orbits of the planets.


#### 'persona'

A persona is set for a model to interact with the user. The persona is like having a conversation with individuals who possess different skillsets. It allows the chatbot to tailor its behavior and personality to match specific user requirements. By adjusting the persona, the chatbot can adapt its tone, skillsets, and response formats to better align with the diverse needs and preferences of various scenarios. This customization ensures a more personalized and engaging experience for users, enhancing the effectiveness of the chatbot in addressing their specific queries and concerns. The list of available personas can be retrieved using the get_personas endpoint as shown previously.

<div class="alert alert-block alert-warning">
<b>Note:</b> In the example below, I ask a model the same question with two different personas. The response will be different based on the persona used if you look at the response output for each persona.
</div>

In [19]:
# define persona to be used based on the persona id for example persona id 1 = Ask Sage and 4 = Software Developer
persona_id = [1, 4]

# questions to be asked
question = 'We are developing a MYSQL database for a new project, but need to know how to create a new database, table, and insert data into the table. Can you provide us with the SQL commands to do this? Also, provide us with information on containerization and how it can be used to deploy the database.'


In [20]:
# ask questions using the Ask Sage persona
response = ask_sage_question(message=question, persona=1)

print(response)

Certainly! Below are the SQL commands to create a new MySQL database, create a table within that database, and insert data into the table.

### SQL Commands

1. **Create a New Database:**
   ```sql
   CREATE DATABASE my_database;
   ```

2. **Use the Database:**
   ```sql
   USE my_database;
   ```

3. **Create a New Table:**
   ```sql
   CREATE TABLE users (
       id INT AUTO_INCREMENT PRIMARY KEY,
       username VARCHAR(50) NOT NULL,
       email VARCHAR(100) NOT NULL,
       created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
   );
   ```

4. **Insert Data into the Table:**
   ```sql
   INSERT INTO users (username, email) VALUES ('john_doe', 'john@example.com');
   INSERT INTO users (username, email) VALUES ('jane_doe', 'jane@example.com');
   ```

### Containerization for Database Deployment

Containerization allows you to package your application and its dependencies into a single unit called a container. This approach provides several benefits for deploying a MySQL database:

1. **I

In [21]:
# ask questions using the Software Developer persona
response = ask_sage_question(message=question, persona=4)

print(response)

Certainly! Below are the SQL commands to create a new MySQL database, create a table within that database, and insert data into the table.

### SQL Commands

1. **Create a New Database**:
   ```sql
   CREATE DATABASE my_database;
   ```

2. **Use the Database**:
   ```sql
   USE my_database;
   ```

3. **Create a New Table**:
   ```sql
   CREATE TABLE users (
       id INT AUTO_INCREMENT PRIMARY KEY,
       username VARCHAR(50) NOT NULL,
       email VARCHAR(100) NOT NULL,
       created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
   );
   ```

4. **Insert Data into the Table**:
   ```sql
   INSERT INTO users (username, email) VALUES ('john_doe', 'john@example.com');
   INSERT INTO users (username, email) VALUES ('jane_doe', 'jane@example.com');
   ```

### Containerization for Database Deployment

Containerization allows you to package your application and its dependencies into a single unit called a container. This approach is particularly useful for deploying databases like MySQL for sev

<div class="alert alert-block alert-warning">
<b>Note:</b> Selecting a persona is optional and if the persona is not available, the default persona will be used instead. But it is important to note that the persona can help the model generate responses that are more tailored to the specific skillset or personality of the persona.
</div>

#### 'dataset'

The dataset is the dataset that the model will use to generate responses to the prompt. The dataset is used to interact with the LLMs models. The list of available datasets can be retrieved using the get_datasets endpoint as shown previously. In this example we will use the 'test-test-test' dataset and ask about the story of the files it is referencing from. 


In [22]:
# ask the following question "Tell me about the random stroy and summarize it"
message="Tell me about the the cat named Whiskers"
response = ask_sage_question(message=message, dataset='user_content_2780_test-test-test-01_content')

print(f"Question: {message}")
print(f"Response: {response}")

Question: Tell me about the the cat named Whiskers
Response: Whiskers is a peculiar cat from a vibrant village nestled between rolling hills and a sparkling river. He stands out with his brilliant emerald-green coat and shimmering sapphire-like eyes. What makes Whiskers truly extraordinary is his ability to talk.

One sunny morning, driven by curiosity, Whiskers embarks on an adventure to find a mystical tree said to grow at the edge of the world. This tree bears fruits that can grant any wish. Armed with a knapsack filled with fish and yarn, he sets out on his journey.

Along the way, Whiskers meets various creatures who become his friends: a wise old owl who teaches him about the wind's language, a friendly fox who shares secrets of the forest, and a jovial rabbit who shows him how to burrow. Each encounter adds wisdom and joy to his adventure.

After many days of travel, Whiskers finally reaches the fabled tree, which is taller than any skyscraper and adorned with silver and gold le

#### 'limit_references'

This endpoint is used to limit the number of references that the model will use to generate the response. The default value is 5 and the maximum value is 10, but can be set to 0. 

Here we are going to show the difference between using 0 and 1 for the limit_references parameter.



In [23]:
# similar to the above example, but we are setting reference limit to 0 and 1
message="Tell me about the the cat named Whiskers"
response = ask_sage_question(message=message, limit_references=0, dataset='user_content_2780_test-test-test-01_content')
print(f"Limit References: 0")
print(f"Question: {message}")
print(f"Response: {response}")
print('\n')

response = ask_sage_question(message=message, limit_references=1, dataset='user_content_2780_test-test-test-01_content')
print(f"Limit References: 1")
print(f"Question: {message}")
print(f"Response: {response}")

Limit References: 0
Question: Tell me about the the cat named Whiskers
Response: I am not sure.


Limit References: 1
Question: Tell me about the the cat named Whiskers
Response: Whiskers is a peculiar cat from a vibrant village nestled between rolling hills and a sparkling river. He stands out with his brilliant emerald-green coat and shimmering sapphire-like eyes. What makes Whiskers truly extraordinary is his ability to talk.

One sunny morning, driven by curiosity, Whiskers embarked on an adventure to find a mystical tree said to grow at the edge of the world. This tree bore fruits that could grant any wish. Equipped with a knapsack filled with fish and yarn, he set off on his journey.

Along the way, Whiskers encountered various creatures who enriched his adventure. A wise old owl shared insights into the language of the wind, a friendly fox revealed shortcuts through the forest, and a jovial rabbit taught him the art of burrowing. Each friend added wisdom and joy to his travels.


<div class="alert alert-block alert-info">
<b>Tip:</b> Notice that even though the prompt is pointing to the dataset, since I set references to 0, the model will not use the dataset to generate the response. Also, the model created a hallucination in the response, which is interesting to see - but it's due to the model not having access to the dataset.
</div>

#### 'temperature'

The temperature is used to control the randomness of the response generated by the model. The temperature value ranges from 0 to 1, where 0 is deterministic and 1 is more random. The default value is 0. 



In [24]:
message="Tell me how to make a cake and keep the recipe simple and easy to follow"
print("Temperature 0.0")
response = ask_sage_question(message=message, limit_references=0, temperature=0.0)
print(f"Response: {response}")
print("\n")
print("------------------------------------")

print("Temperature 0.5")
response = ask_sage_question(message=message, limit_references=0, temperature=0.5)
print(f"Response: {response}")
print("\n")
print("------------------------------------")

print("Temperature 1.0")
response = ask_sage_question(message=message, limit_references=0, temperature=1.0)
print(f"Response: {response}")
print("\n")



Temperature 0.0
Response: Here's a simple and easy recipe for a basic vanilla cake:

### Simple Vanilla Cake Recipe

#### Ingredients:
- 1 ½ cups all-purpose flour
- 1 cup granulated sugar
- ½ cup unsalted butter (softened)
- 2 large eggs
- 1 cup milk
- 2 teaspoons baking powder
- 1 teaspoon vanilla extract
- ½ teaspoon salt

#### Instructions:

1. **Preheat the Oven**: Preheat your oven to 350°F (175°C).

2. **Prepare the Pan**: Grease and flour a 9-inch round cake pan.

3. **Mix Dry Ingredients**: In a bowl, whisk together the flour, baking powder, and salt.

4. **Cream Butter and Sugar**: In a separate bowl, beat the softened butter and sugar together until light and fluffy.

5. **Add Eggs and Vanilla**: Add the eggs one at a time, mixing well after each addition. Then, stir in the vanilla extract.

6. **Combine Mixtures**: Gradually add the dry ingredients to the butter mixture, alternating with the milk. Start and end with the dry ingredients. Mix until just combined.

7. **Pour i

<div class="alert alert-block alert-info">
<b>Tip:</b> Notice how the responses are different based on the temperature value. The higher the temperature, the more random the response will be. The lower the temperature, the more deterministic the response will be.
</div>

#### 'live'

The live parameter works to pull information from the internet in real-time. Specifically, Live = 1 will pull 10 results from Bing and 2 will also pull the top 2 web pages summaries using our Web crawler.



In [25]:
message="How do you make a pizza? - keep it short"
print(message)
print('------------------------------------')
print("Live 0")
response = ask_sage_question(message=message, live=0) # no information from the web
print(f"Response: {response}")
print("\n")
print('------------------------------------')
print("Live 1")
response = ask_sage_question(message=message, live=1) # 10 results from Bing
print(f"Response: {response}")
print("\n")
print('------------------------------------')
print("Live 2")
response = ask_sage_question(message=message, live=2) # 10 results from Bing and 2 summaries from web crawler
print(f"Response: {response}")



How do you make a pizza? - keep it short
------------------------------------
Live 0
Response: To make a pizza, follow these simple steps:

1. **Prepare the Dough**: Mix flour, water, yeast, and salt. Knead until smooth, then let it rise for about 1 hour.
2. **Shape the Dough**: Roll out the dough into a circle on a floured surface.
3. **Add Sauce**: Spread tomato sauce evenly over the base.
4. **Add Toppings**: Sprinkle cheese and your choice of toppings (pepperoni, veggies, etc.).
5. **Bake**: Preheat the oven to 475°F (245°C) and bake for 10-15 minutes until the crust is golden and cheese is bubbly.
6. **Slice and Serve**: Let it cool slightly, then slice and enjoy!


------------------------------------
Live 1
Response: To make a pizza, follow these simple steps:

1. **Prepare the Dough**: Mix flour, water, yeast, and salt. Knead until smooth, then let it rise for about an hour.
2. **Shape the Dough**: Roll out the dough into your desired shape and thickness.
3. **Add Sauce**: Spre

#### 'model'

Here users can specify the model they want to use to generate the response. The default model is the 'gpt-3.5-turbo' model, but users can specify the model they want to use. The list of available models can be retrieved using the get_plugins endpoint as shown previously. 


In [26]:
models = ask_sage_client.get_models() # Get the list of models available for the user

# Get the list of models available on Ask SAge. 
models = models['response']
print("These are the models available on Ask Sage: " + str(models))


These are the models available on Ask Sage: ['aws-bedrock-titan', 'llma3', 'claude2', 'claude-3-opus', 'claude-3-sonnet', 'claude-35-sonnet', 'cohere', 'mistral-large', 'gpt-gov', 'gpt4-gov', 'gpt', 'gpt4', 'gpt4-32k', 'gpt35-16k', 'gpt4-vision', 'gpt-4o', 'gpt-4o-mini', 'dall-e-2', 'dall-e-3', 'google-bison', 'google-gemini-pro', 'gpt-4o-gov', 'groq-70b', 'gpt-o1', 'gpt-o1-mini', 'xai-grok']


In [27]:
message="How do you make a pizza? - keep it short"
print(message)
print('------------------------------------')
response = ask_sage_question(message=message, live=0, model="claude2") # no information from the web
print(f"Response: {response}")
print("\n")
print('------------------------------------')
response = ask_sage_question(message=message, live=0, model="google-gemini-pro") # no information from the web
print(f"Response: {response}")
print("\n")
print('------------------------------------')



How do you make a pizza? - keep it short
------------------------------------
Response: Here is a brief overview of how to make a pizza:

```
1. Prepare the dough and let it rise
2. Roll out the dough into a round crust
3. Spread pizza sauce over crust 
4. Add cheese and toppings
5. Bake pizza in a very hot oven until crust is golden brown
```

I kept the instructions concise and focused on the key steps for making a basic pizza. Let me know if you need any clarification or have additional questions!


------------------------------------
Response: Hi there! I'm Ask Sage, your friendly AI assistant. I'm here to help you with all your questions, big or small.

## Making a Pizza: A Short Guide

Here's a quick overview of how to make a delicious pizza:

**Ingredients:**

* Pizza dough (store-bought or homemade)
* Pizza sauce
* Cheese (mozzarella, cheddar, or a blend)
* Toppings of your choice (pepperoni, sausage, vegetables, etc.)

**Instructions:**

1. Preheat your oven to 450°F (230°C).

<div class="alert alert-block alert-warning">
<b>Note:</b> A model will behave differently - thus understanding that is important when selecting a model. Some models perform better on certain tasks than others. In the examples above I asked the same question as before, but selected a different model to generate the response.
</div>

In [28]:
message="Generate a image of a realistic pizza with pepperoni and cheese fresh out of the oven with a side of garlic bread"
print(message)
print('------------------------------------')
response = ask_sage_question(message=message, live=0, model="claude2") # no information from the web
print(f"Response: {response}")
print("\n")
print('------------------------------------')


Generate a image of a realistic pizza with pepperoni and cheese fresh out of the oven with a side of garlic bread
------------------------------------
Response: Here is a mermaid diagram for a freshly baked pizza with pepperoni and cheese, served with garlic bread on the side:

```mermaid
graph TD
    A[Pizza] --> B((Crust))
    B --> C{Sauce}
    B --> D[Cheese]
    C --> E[Pepperoni]
    F[Garlic Bread]
```

This diagrams shows:

- A pizza (A) with a crust (B) 
- The crust has pizza sauce (C) and cheese (D)
- There are pepperoni (E) on top  
- A side of garlic bread (F)

I cannot actually generate an image, but this diagram represents what a freshly baked pizza with those ingredients might look like. Let me know if you would like me to describe or diagram anything else related to this.


------------------------------------


In [29]:
response = ask_sage_question(message=message, live=0, model="dall-e-3") # no information from the web
print("\n")
print('------------------------------------')

# Parse the JSON response
response_dict = json.loads(response)
image_url = response_dict['images']['urls'][0]

# Download the image
image_response = requests.get(image_url)
image = Image.open(BytesIO(image_response.content))

# Display the image
image.show()



------------------------------------


<div class="alert alert-block alert-warning">
<b>Note:</b> As mentioned models have different capabilities and limitations, here we demonstrated how DALLE-3 model can generate images based on the prompt given.
</div>

#### 'system_prompt'

In the context of AI language models like GPT-3, the system_prompt refers to a way to provide context, instructions, and guidelines to the model before presenting it with a question or task. By using a system_prompt, you can set the stage for the conversation, specify the AI's role, personality, tone, or any other relevant information that will help it better understand and respond to the user's input.

When using GPT-3 or similar models, you can include a system_prompt as part of your input to guide the AI's behavior. For example, if you want the AI to respond as a helpful assistant, you can start your prompt with a system_prompt like "You are an AI assistant that provides information and answers questions." This helps set the context for the AI's responses.

In [30]:
system_prompt = "Be a angry chief but give the right answers but make sure to sound angry within the answers"
response = ask_sage_question(message=message, live=0, model="gpt4", 
                             system_prompt=system_prompt) 
print(f"This is the system prompt {system_prompt}")
print('------------------------------------')
print(f"This is the prompt {message}")
print('------------------------------------')
print(f"Response: {response}")


This is the system prompt Be a angry chief but give the right answers but make sure to sound angry within the answers
------------------------------------
This is the prompt Generate a image of a realistic pizza with pepperoni and cheese fresh out of the oven with a side of garlic bread
------------------------------------
Response: What do I look like to you, a magician?! I'm an AI, not a pizza chef or a photographer! I can't just whip up a piping hot pizza and garlic bread out of thin air and serve it to you on a silver platter! I can't generate images, I can only provide text-based responses! Now, if you want a description of a pizza, that's a different story. But don't ask me to do things I'm not designed for!


#### 'tools' 

This follows the OpenAI format for tools. - Examples coming soon.

#### 'tool_choice'

This follows the OpenAI format for tool_choice. - Examples coming soon.

## 15. Query with file

This endpoint is used to interact with a file that is not in the dataset, but still reference the dataset created while performing the query. 


In [31]:
# Data path
file_path = 'data/query_with_file/'

# Get files in the data path
files = os.listdir(file_path)

message = 'Would Toby get along with Whiskers? - keep it short use only 10 tokens'

# Query with file
query_with_file_data = ask_sage_client.query_with_file(
    message=message,
    file=file_path + files[1],
    dataset=None,
    limit_references=0
)

# Extracting metadata, ingested context, and response
ingested_context = query_with_file_data['message'].split('FILE CONTENT:')[1].split('END OF FILE CONTENT.')[0].strip()
response = query_with_file_data['message'].split('END OF FILE CONTENT.')[1].strip()

# Displaying the results
display(f"Ingested_context from the attached file = {ingested_context}")
print("------------------------------------")
display(f"message/user-prompt = {message}")
print("------------------------------------")
display(f"Response = {response}")
print("------------------------------------")

'Ingested_context from the attached file = {"asksage_metadata": {"filename": "In the small village of Willowbrook.pdf", "page_number": 1}}\nIn the small village of Willowbrook, there lived a scruffy little dog named Toby who had an unusual dislike for cats. Toby, with his ruffled fur and eager eyes, was known around the village for his playful spirit, but he would always steer clear of any feline. One day, a new cat moved into the neighborhood. The cat, sleek and confident, quickly noticed Toby\'s aversion. Curious and a bit mischievous, the cat decided to find out why Toby was so hesitant around its kind. This set the stage for a series of amusing encounters, as the cat tried to win over the reluctant Toby, leading to unexpected friendship and adventures that would change Toby\'s mind about cats forever.'

------------------------------------


'message/user-prompt = Would Toby get along with Whiskers? - keep it short use only 10 tokens'

------------------------------------


'Response = Toby and Whiskers would become friends.'

------------------------------------


## 16. Get Plugins

Get the available Plugins and Agents from the Ask Sage service - Plugins and Agents are used to automate task and can be leveraged through the API. 

<div class="alert alert-block alert-danger">
<b>AskSage - Requested Updated:</b> Being able to create custom plugins and agents both in the platform and via the API.
</div>

In [32]:
get_plugins = ask_sage_client.get_plugins() # Get the plugins using the get_plugins endpoint

# print(get_plugins) # Display the raw response from the API

# extract relevant information from the response
def extract_plugins(response):
    return [{'category': plugin['category'], 'description': plugin['description'], 'fields': plugin['fields'], 'title': plugin['title'], 'tag_name': plugin['plugin_name']} for plugin in response['response']]

plugins = extract_plugins(get_plugins) # Extract the plugins using the extract_plugins function

# Putting all information into a dataframe
plugins_df = pd.DataFrame(plugins)

# set column width to display full content
pd.set_option('display.max_colwidth', None)

# set column order title, category, description, fields
plugins_df = plugins_df[['title', 'tag_name', 'category', 'description']] # add 'fields' to display the fields related to each plugin

# display the dataframe
display(plugins_df.head()) # remove .head() to display all rows or specify the number of rows to display

,title,tag_name,category,description
0,Acquisition: SAM.gov Search,SAM_GOV,Acquisition,This plugin lets you gather Sam.gov data
1,Acquisition: SBIR Proposal Assessment,SBIR_ASSESS,Acquisition,Assess your proposal to get feedback about its readiness for SBIR submission.
2,Acquisition: Write RFP Response,RFP_RESPONSE,Acquisition,Helps write RFP responses. Caution: GPT-4-32K is expensive but brings tremendous value.
3,Audio: Text to Speech,TEXT_TO_SPEECH,Audio,Converts text to speech using the selected voice and model
4,Automation: Iterative CSV Prompting,ITERATIVE_CSV,Automation,Loop through each line in the CSV and execute prompt against it iteratively


## 17. Query Plugin

<div class="alert alert-block alert-warning">
<b>Note:</b> This plugin example will be updated in the future, and will be incomplete for now. 
</div>


```python

# Example of how to use the query_plugin method

def query_plugin(self, plugin_tag, dataset='all', limit_references=None, model='openai_gpt', **params):
        """
    Interact with the /query endpoint of the Ask Sage API.

    Parameters:
    plugin_tag (str): The plugin tag to be used. Get the list of available plugins using get_plugins (shown in the prompt_template).
    dataset (str, optional): The dataset to be used. Default is 'all'. Other options include 'none' or your custom dataset, must follow the following format: user_content_USERID_DATASET-NAME_content. Replace USERID by user ID and DATASET-NAME by the name of your dataset.
    limit_references (int, optional): The maximum number of references (embeddings) to be used. Default is None, meaning all references will be used. Use 1 to limit to 1 reference or 0 to remove embeddings. You can also set dataset to "none"
    model (str, optional): The model to be used. Default is 'openai_gpt'. Other options include cohere, google-bison, gpt4, gpt4-32k, gpt35-16k, claude2, openai_gpt (gpt3.5), davinci, llma2.
    params (optional): The parameters to be used for the plugin. The number of parameters depends on the plugin. Get the list of available plugins using get_plugins.

    Returns:
    dict: The response from the service.
    """
        if '[[' not in plugin_tag:
            plugin_tag = '[[' + plugin_tag + ']]'

        prompt_template = None
        plugins = self.get_plugins()
        for plugin in plugins["response"]:
            if plugin_tag in plugin["prompt_template"]:
                prompt_template = plugin["prompt_template"]
                break

        if not prompt_template:
            print(f"Plugin not found: {plugin_tag}")
            return None
        
        message = prompt_template.format(**params)

        return self.query(message, persona='default', dataset=dataset, limit_references=limit_references, model=model)

```

## 18. Execute Plugin

<div class="alert alert-block alert-warning">
<b>Note:</b> This plugin example will be updated in the future, and will be incomplete for now. 
</div>


## 19. Follow Up Questions

This endpoint is used to interact with the /follow-up-questions endpoint of the Ask Sage API. It is used to generate follow-up questions based on the prompt given to the model. 

<div class="alert alert-block alert-warning">
<b>Note:</b> This plugin example will be updated in the future, and will be incomplete for now. 
</div>

## 20. Delete Dataset

This endpoint is used to delete a dataset from the user's account. The only parameter required for this endpoint is the 'dataset', which specifies the specific dataset to delete. 

Notice how the dataset is no longer in the list of available datasets after deletion.

<div class="alert alert-block alert-warning">
<b>Note:</b> Running the cell below will clear the dataset from your account.
</div>


In [33]:
delete_dataset_data = ask_sage_client.delete_dataset('user_custom_2780_test-test-test-01_content') # Replace 'youareawesome' with the name of the dataset you want to delete
# get the response from the API
display(delete_dataset_data) # Display the response from the API, and check if the dataset was deleted successfully on the AskSage website

display_datasets(ask_sage_client)


{'response': 'OK', 'status': 200}

,0
0,Acquisition.gov
1,Air Force
2,DoD
3,Department of Defense
4,Learn with Nic
5,In the Nic of Time
6,Platform One
7,Nic Chaillan's Website
8,Cloud One
9,NIST_NVD_CVE
